In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
      rescale=1./255.,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'F:\\mask detector\\images\\train',  # This is the source directory for training images
        target_size=(224, 224),  #images will be resized to size 224x224
        batch_size=64,
        class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1./255.)

validation_generator = validation_datagen.flow_from_directory(
        'F:\\mask detector\\images\\validation',  # This is the source directory for training images
        target_size=(224, 224),  #images will be resized to size 224x224
        batch_size=32,
        class_mode='binary')

Found 1376 images belonging to 2 classes.
Found 142 images belonging to 2 classes.


In [2]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')

In [3]:
base_model.trainable = False #transfer learning
#we take the weights of the pre trained model as it is

In [5]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    #tf.keras.layers.Dense(32, activation = 'relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                81984     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 2,340,033
Trainable params: 82,049
Non-trainable params: 2,257,984
_________________________________________________________________


In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.003), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [9]:
history = model.fit(train_generator, epochs = 5, validation_data = (validation_generator))

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 22 steps, validate for 5 steps
Epoch 1/5
22/22 [==============================] - 102s 5s/step - loss: 0.1215 - accuracy: 0.9455 - val_loss: 0.0098 - val_accuracy: 1.0000
Epoch 2/5
22/22 [==============================] - 99s 5s/step - loss: 0.0914 - accuracy: 0.9622 - val_loss: 0.0309 - val_accuracy: 0.9859
Epoch 3/5
22/22 [==============================] - 100s 5s/step - loss: 0.1060 - accuracy: 0.9629 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 4/5
22/22 [==============================] - 101s 5s/step - loss: 0.0946 - accuracy: 0.9593 - val_loss: 0.0079 - val_accuracy: 1.0000
Epoch 5/5
22/22 [==============================] - 101s 5s/step - loss: 0.0920 - accuracy: 0.9658 - val_loss: 0.0880 - val_accuracy: 0.9648


In [10]:
import cv2
import numpy as np
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('C:\\Users\\ahuja\\Downloads\\opencv-master\\data\\haarcascades\\haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(224,224))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,224,224,3))
        reshaped = np.vstack([reshaped])
        result=model1.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()